# Fine-tuning

Fine-tuning refers to the process in transfer learning in which the parameter values of a model trained on a large dataset are modified when the training process continues on a small dataset (see [Kevin Murphy's book](https://probml.github.io/pml-book/book1.html) Section 19.2 for further details). The main motivation is to adapt a pre-trained model trained on a large amount of data to tackle a specific task providing better performance that would be achieved training on the small task-specific dataset.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

Taking a look at the content of the Europarl-ST dataset having English as a source language

In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("tj-solergibert/Europarl-ST-processed-mt-en")
print(raw_datasets)

The original dataset includes the translation from English into several European languages

In [ ]:
raw_datasets["train"].features

{'source_text': Value(dtype='string', id=None),
 'dest_text': Value(dtype='string', id=None),
 'dest_lang': ClassLabel(names=['de', 'en', 'es', 'fr', 'it', 'nl', 'pl', 'pt', 'ro'], id=None)}

Let us take a look at the translations of the first English sentence

In [ ]:
raw_datasets["train"][:7]["source_text"]

['Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.',
 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.']

In [ ]:
raw_datasets["train"][:7]["dest_text"]

['Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.',
 'La mayoría de los servicios financieros, incluidos los seguros y la gestión de fondos de inversión, están exentos de IVA desde 1977.',
 'Depuis 1977, la plupart des services financiers, dont les assurances et la gestion des fonds de placement, ne sont pas tenus d ’ appliquer une TVA.',
 'Dal 1997 la maggior parte dei servizi finanziari, compresi i servizi assicurativi e la gestione di fondi di investimento, sono esenti da IVA.',
 'Sinds 1977 zijn de meeste financiële diensten, met inbegrip van verzekeringen en het beheer van beleggingsfondsen, vrijgesteld van btw.',
 'większość usług finansowych, w tym usług w zakresie ubezpieczeń i zarządzania funduszami inwestycyjnymi, była zwolniona z opodatkowania podatkiem VAT.',
 'Desde 1977 que a maioria dos serviços financeiros, incluindo os seguros e a gestão de fundos de investimento,

In [ ]:
raw_datasets["train"][:7]["dest_lang"]

[0, 2, 3, 4, 5, 6, 7]

Filtering Europarl-ST only for English into Spanish using a simple [lambda function](https://realpython.com/python-lambda/) with the [Dataset.filter() function](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.filter) and taking a small sample with [Dataset.select()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.select) functions together

In [ ]:
lang="de"
lang_id = raw_datasets["train"].features["dest_lang"].names.index(lang)
raw_datasets["train"] = raw_datasets["train"].filter(lambda x: x["dest_lang"] == lang_id).select(range(1024))
raw_datasets["valid"] = raw_datasets["valid"].filter(lambda x: x["dest_lang"] == lang_id).select(range(128))
raw_datasets["test"] = raw_datasets["test"].filter(lambda x: x["dest_lang"] == lang_id).select(range(128))

  0%|          | 0/603 [00:00<?, ?ba/s]

  0%|          | 0/82 [00:00<?, ?ba/s]

  0%|          | 0/87 [00:00<?, ?ba/s]

A good practice is to take a small random sample to get a quick feel for the type of data you’re working with. In Datasets, we can create a random sample by using the [Dataset.shuffle()](https://huggingface.co/docs/datasets/v2.9.0/en/package_reference/main_classes#datasets.Dataset.shuffle):

In [ ]:
raw_sample = raw_datasets["train"].shuffle(seed=13).select(range(5))
raw_sample[:5]

{'source_text': ['During this period, two problems have essentially arisen: the definition of the scope of the exemption and the impossibility of recovering VAT incurred in order to provide exempt services, giving rise to the phenomenon of hidden VAT.',
  'They must also ensure adherence at home to the principle of respect for human rights and the rule of law which President Medvedev himself proclaimed at the start of his term of office.',
  'The euro cannot simply act as a safety anchor, but must also be an engine that can drive growth. The euro area and Economic and Monetary Union must be capable of responding to these challenges.',
  'To end, I must say to you, Madam Vice-President, that you already know that you can count on the support of this Parliament.',
  'starts the negotiations with a view to concluding an agreement or partnership with Russia without that country fully complying with and respecting the agreements that it signed with the European Union on 12 August and 8 Sept

Now we load the pre-trained tokenizer for the mT5 model and apply it to a sample English-Spanish pair

In [ ]:
from transformers import AutoConfig, AutoTokenizer

checkpoint = "t5-small"
config = AutoConfig.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

/usr/local/lib/python3.8/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
input_ids = tokenizer(raw_datasets["train"][0]["source_text"])
print(input_ids)
labels = tokenizer(raw_datasets["train"][0]["dest_text"]).input_ids
print(labels)

{'input_ids': [1541, 16433, 6, 167, 981, 364, 6, 379, 958, 11, 1729, 3069, 758, 6, 43, 118, 1215, 9045, 45, 19569, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
[8709, 16433, 3163, 67, 7957, 15156, 7691, 24150, 6, 236, 13259, 3, 24591, 35, 64, 18622, 193, 9682, 7610, 7, 6, 193, 74, 29914, 11199, 403, 7026, 5, 1]


In [ ]:
print(tokenizer.convert_ids_to_tokens(labels))

['▁Seit', '▁1977', '▁wurden', '▁die', '▁meisten', '▁Finanz', 'dienst', 'leistungen', ',', '▁ein', 'schließlich', '▁', 'Versicherung', 'en', '▁und', '▁Verwaltung', '▁von', '▁Investment', 'fund', 's', ',', '▁von', '▁der', '▁Mehrwert', 'steuer', '▁aus', 'genommen', '.', '</s>']


In [ ]:
tokenizer.decode(labels)

'Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.</s>'

We can apply the tokenizer function to any dataset taking advantage that Hugging Face Datasets are [Apache Arrow](https://arrow.apache.org) files stored on the disk, so you only keep the samples you ask for loaded in memory.

To keep the data as a dataset, we will use the Dataset.map() method. This also allows us some extra flexibility, if we need more preprocessing done than just tokenization. The map() method works by applying a function on each element of the dataset, so let us define a function that tokenizes our inputs:

In [ ]:
max_input_length = 40
max_dest_length = 40

def tokenize_function(sample):
    task_prefix = "translate English to German: "
    inputs = [task_prefix + s for s in sample["source_text"]]
    model_inputs = tokenizer(inputs,max_length=max_input_length,truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        model_inputs['labels'] = tokenizer(sample["dest_text"],max_length=max_dest_length,truncation=True).input_ids
    return model_inputs

In [ ]:
tokenized_sample = tokenize_function(raw_datasets["train"][:5])
print(tokenized_sample)

{'input_ids': [[13959, 1566, 12, 2968, 10, 1541, 16433, 6, 167, 981, 364, 6, 379, 958, 11, 1729, 3069, 758, 6, 43, 118, 1215, 9045, 45, 19569, 5, 1], [13959, 1566, 12, 2968, 10, 3, 2092, 48, 1059, 6, 192, 982, 43, 3, 8317, 7931, 29, 10, 8, 4903, 13, 8, 7401, 13, 8, 20554, 11, 8, 256, 2748, 7, 11102, 13, 21827, 19569, 3, 20890, 16, 455, 1], [13959, 1566, 12, 2968, 10, 3699, 2121, 6, 1611, 981, 5660, 11, 512, 16690, 6, 84, 43, 4161, 8, 5102, 11, 25367, 13, 2673, 12, 370, 175, 364, 6, 43, 856, 974, 12, 8, 3, 30556, 5, 1], [13959, 1566, 12, 2968, 10, 100, 934, 19, 8, 166, 3332, 12, 2270, 3, 9, 21130, 84, 6, 16, 811, 12, 271, 15266, 3, 104, 1374, 12, 8, 7897, 13, 8, 1611, 2243, 13, 6923, 3, 104, 19, 29451, 1], [13959, 1566, 12, 2968, 10, 27, 241, 12, 3, 24615, 15, 8, 5346, 1238, 6, 1363, 6887, 2138, 6, 30, 112, 1287, 161, 16, 5874, 48, 934, 30, 46, 962, 84, 19, 78, 24367, 6280, 11, 18155, 1561, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

The way the Datasets library applies this processing is by adding new fields to the datasets, one for each key in the dictionary returned by the tokenize function:

In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/1 [00:00<?, ?ba/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    test: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 128
    })
    train: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1024
    })
    valid: Dataset({
        features: ['source_text', 'dest_text', 'dest_lang', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 128
    })
})

The function that is responsible for putting together samples inside a batch is called a collate function. It is an argument you can pass when you build a DataLoader, the default being a function that will just convert your samples to PyTorch tensors and concatenate them. This is not possible in our case since the inputs we have are not all of the same size. We have deliberately postponed the padding, to only apply it as necessary on each batch and avoid having over-long inputs with a lot of padding. 

To do this in practice, we have to define a collate function that will apply the correct amount of padding to the items of the dataset we want to batch together. Fortunately, the Transformers library provides us with such a function via DataCollatorForSeq2Seq It takes a tokenizer when you instantiate it (to know which padding token to use, and whether the model expects padding to be on the left or on the right of the inputs) and will do everything you need:

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--t5-small/snapshots/3479082dc36f8a4730936ef1c9b88cd8b0835c53/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
   

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

To take a look at how this function works, now we have our dataset encoded, let us select only those keys that will be needed to create the batch discarding the textual features.

In [ ]:
samples = [tokenized_datasets["train"][i] for i in range(2)]
print(samples)
for i in range(len(samples)):
    samples[i] = {key: value for key, value in samples[i].items() if key not in ["source_text", "dest_text", "dest_lang"]}
print(samples)

[{'source_text': 'Since 1977, most financial services, including insurance and investment fund management, have been exempt from VAT.', 'dest_text': 'Seit 1977 wurden die meisten Finanzdienstleistungen, einschließlich Versicherungen und Verwaltung von Investmentfunds, von der Mehrwertsteuer ausgenommen.', 'dest_lang': 0, 'input_ids': [13959, 1566, 12, 2968, 10, 1541, 16433, 6, 167, 981, 364, 6, 379, 958, 11, 1729, 3069, 758, 6, 43, 118, 1215, 9045, 45, 19569, 5, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [8709, 16433, 3163, 67, 7957, 15156, 7691, 24150, 6, 236, 13259, 3, 24591, 35, 64, 18622, 193, 9682, 7610, 7, 6, 193, 74, 29914, 11199, 403, 7026, 5, 1]}, {'source_text': 'During this period, two problems have essentially arisen: the definition of the scope of the exemption and the impossibility of recovering VAT incurred in order to provide exempt services, giving rise to the phenomenon of hidden VAT.', 'dest_text

In [ ]:
batch = data_collator(samples)
print(batch)

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[13959,  1566,    12,  2968,    10,  1541, 16433,     6,   167,   981,
           364,     6,   379,   958,    11,  1729,  3069,   758,     6,    43,
           118,  1215,  9045,    45, 19569,     5,     1,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [13959,  1566,    12,  2968,    10,     3,  2092,    48,  1059,     6,
           192,   982,    43,     3,  8317,  7931,    29,    10,     8,  4903,
            13,     8,  7401,    13,     8, 20554,    11,     8,   256,  2748,
             7, 11102,    13, 21827, 19569,     3, 20890,    16,   455,     1]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]), 'labels': tensor([[ 8709, 16433,  3163,    67,  7957

## Evaluation

The last thing to define for our Seq2SeqTrainer is how to compute the metrics from the predictions. We need to define a function for this, which will just use the metric we loaded earlier, and we have to do a bit of pre-processing to decode the predictions into texts:

In [ ]:
!pip install sacrebleu
from evaluate import load

metric = load("sacrebleu")
dest_preds = ["Esto es un ejemplo de cálculo de BLEU.", "Esto es otro."]
dest_labels = [["Este es un ejemplo de cálculo de BLEU."], ["Este es otro."]]
metric.compute(predictions=dest_preds, references=dest_labels)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 11.5 MB/s eta 0:00:00


{'score': 78.75110621102682,
 'counts': [11, 9, 7, 5],
 'totals': [13, 11, 9, 7],
 'precisions': [84.61538461538461,
  81.81818181818181,
  77.77777777777777,
  71.42857142857143],
 'bp': 1.0,
 'sys_len': 13,
 'ref_len': 13}

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

Evaluation of the pretrained model:

In [ ]:
task_prefix = "translate English to German: "
inputs = tokenizer([task_prefix + sentence for sentence in tokenized_datasets["test"]["source_text"]], max_length=max_input_length, truncation=True, return_tensors="pt", padding=True)
output_sequences = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
result = compute_metrics((output_sequences,tokenized_datasets["test"]["labels"]))
print(f'BLEU score: {result["bleu"]}')

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



BLEU score: 18.2408


## Training

The first step before we can define our Trainer is to define a TrainingArguments class that will contain all the hyperparameters the Trainer will use for training and evaluation. The only argument you have to provide is a directory where the trained model will be saved, as well as the checkpoints along the way. For all the rest, you can leave the defaults, which should work pretty well for a basic fine-tuning.

In [ ]:
from transformers import Seq2SeqTrainingArguments

batch_size = 16
model_name = checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-en-to-de",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
    predict_with_generate=True,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


This is because mT5 has not been pretrained on translation, so the head of the pretrained model has been discarded and a new head suitable for sequence classification has been added instead. The warnings indicate that some weights were not used (the ones corresponding to the dropped pretraining head) and that some others were randomly initialized (the ones for the new head). It concludes by encouraging you to train the model, which is exactly what we are going to do now.

Once we have our model, we can define a Trainer by passing it all the objects constructed up to now — the model, the training_args, the training and validation datasets, our data_collator, and our tokenizer:

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


To fine-tune the model on our dataset, we just have to call the train() method of our Trainer:

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dest_lang, source_text, dest_text. If dest_lang, source_text, dest_text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1024
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 320
  Number of trainable parameters = 60506624


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.118580,16.792800,18.656200
2,No log,1.096502,16.984100,18.656200


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: dest_lang, source_text, dest_text. If dest_lang, source_text, dest_text are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 128
  Batch size = 16
Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}

Generate config GenerationConfig {
 

KeyboardInterrupt: ignored

## Inference

At inference time, it is recommended to use [generate()](https://huggingface.co/docs/transformers/v4.26.1/en/main_classes/text_generation#transformers.GenerationMixin.generate). This method takes care of encoding the input and feeding the encoded hidden states via cross-attention layers to the decoder and auto-regressively generates the decoder output. Check out [this blog post](https://huggingface.co/blog/how-to-generate) to know all the details about generating text with Transformers. There’s also [this blog post](https://huggingface.co/blog/encoder-decoder#encoder-decoder) which explains how generation works in general in encoder-decoder models.

In [ ]:
task_prefix = "translate English to German: "
inputs = tokenizer([task_prefix + sentence for sentence in tokenized_datasets["test"]["source_text"]], max_length=max_input_length, truncation=True, return_tensors="pt", padding=True)
output_sequences = model.generate(input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"])
result = compute_metrics((output_sequences,tokenized_datasets["test"]["labels"]))
print(f'BLEU score: {result["bleu"]}')

Generate config GenerationConfig {
  "decoder_start_token_id": 0,
  "eos_token_id": 1,
  "pad_token_id": 0,
  "transformers_version": "4.26.1"
}



BLEU score: 17.9905
